# MY NOTEBOOK

## CONFIG

In [1]:
%load_ext autoreload
%autoreload

### Imports

In [2]:
import os
import fnmatch
import re
import random
import time
import sys
import warnings

import librosa
import tensorflow as tf

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt 

import itertools

#import progressbar
from tqdm import tqdm_notebook as tqdm
from time import sleep

from models import *

## Constants

### Local Machine

In [3]:
MTT_DIR = "../MTT/"
DATA_DIRECTORY = MTT_DIR + "dataset/"
WAV_DIRECTORY = MTT_DIR + "wav-dataset/"

# CSV containing song details (title, artist, id, mp3_path,...)
CLIP_INFO_FINAL = "clip_info_final.csv"
# CSV containg the labels (clip_id, labels, mp3_path)
ANNOTATIONS_FINAL = "annotations_final.csv"
# CSV : what is it useful for ?
COMPARISONS_FINAL = "comparisons_final.csv"
LABELS_FILE = MTT_DIR + ANNOTATIONS_FINAL

SUB_DIRS = "0123456789abcdef"

LOGDIR = "checkpoints/"

AUDIO1_path = "../MTT/mtt_data_mp3.zip/0/american_bach_soloists-j_s__bach_\
_transcriptions_of_italian_music-02-concerto_in_a_minor_for_four_harpsichords\
_bwv_1065_ii_largo-88-117.mp3"

### LTS2 Server

This cell defines all the needed data paths, when running on LTS2 server.
Don't run this cell if you are not running the jupyter notebook on the LTS2 server ! (will overwrite the variables defined on the cell above).

You can either create a cell with your own paths, or modify the cell above with your custom paths.

In [ ]:
MTT_DIR = "/mnt/scratch/students/jjgweber-MagnaTagATune/"
DATA_DIRECTORY = MTT_DIR + "dataset/"
WAV_DIRECTORY = MTT_DIR + "wav-dataset/"

# CSV containg the labels (clip_id, labels, mp3_path)
ANNOTATIONS_FINAL = "annotations_final.csv"

LABELS_FILE = MTT_DIR + ANNOTATIONS_FINAL

# need to create this directory on the server !!
LOGDIR = "checkpoints/"

SUB_DIRS = "0123456789abcdef"

### Data

In [4]:
FILE_NB = 25880 #25863
FILE_LENGTH = 465984
# 465984 = 2 × 2 × 2 × 2 × 2 × 2 × 3 × 3 × 809
# useful for batches > for now divide by 9 (instead of 10)

BATCH_NB = 9
BATCH_SIZE = int(FILE_LENGTH/BATCH_NB)
SAMPLE_SIZE = 0
SAMPLE_RATE = 16000
RECEPTIVE_FIELD = 0

TRAIN_DIR = "0123456789abcde"
TEST_DIR ="f"

BASIC_CONFIG ={'numOutputNeurons':500}
LEARNING_RATE = 0.001

### Mutable parameters

In [5]:
# Variables which are often modified to test the algorithm
NB_SONGS = 20
EPOCHS = 50 # check in paper
LABELS_NAME = ['guitar', 'techno']
NB_LABELS = len(LABELS_NAME)

## Loading tool functions

In [6]:
def load_labels(labels_file_name):
    pd.read_csv(labels_file_name)

def randomize_files(files):
    for file in files:
        file_index = random.randint(0, (len(files) - 1))
        yield files[file_index]


# ATTENTION : in the current configuration, everytime this function is called it goes through 
# all the sub-directories even if the sample is set to 6
# it only prunes the output at the end
# > there could be a more efficient way for doing this
# TODO later
def find_files(directory, pattern='*.mp3', sample=None, sub_dir=None):
    '''Recursively finds all files matching the pattern.'''
    # subdir sould be a string, for example "abc03", 
    # meaning we take data from directories a,b,c,0 and 3
    
    files = []
    directories = []
    
    # TODO : add lines to check format of input
    # TODO : try/except ?
    
    if sub_dir!=None :
        for c in sub_dir :
            directories.append(directory+c+"/")
    else :
        directories.append(directory)
    
    for path in directories :
        for root, dirnames, filenames in os.walk(path):
            for filename in fnmatch.filter(filenames, pattern):
                files.append(os.path.join(root, filename))

    if sample!=None :
        try:
            return files[:sample]
        except TypeError:
            print("Argument sample should be either None, or an integer :\
             the number of first n samples to take.")
    else :
        return files

## Prediction labels

### Load and clean

In [7]:
start = time.time()
labels = pd.read_csv(LABELS_FILE, sep = '"\t"')
end = time.time()
print("Duration : {:.3}".format(end-start))

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


Duration : 2.25


In [8]:
# Prepare header to put back in the end
# remove quotes and take all columns except the first one
header = list(map(lambda x : x.replace('"', ''), labels))[1:]
# add back the first column, separated in two
header = ['clip_id', 'no_voice']+header
# create dictionary
header = dict(enumerate(header))

In [9]:
# Solve format problem : two first columns are merged
# extract first column and rest
left, right = labels['"clip_id\t""no voice"'], labels.iloc[:, 1:]
# split first column in two part at separator "\t"
split = left.str.split(pat = "\t", expand=True).replace('"', '')

# put back the first column which is now two, with the rest
cleaned = pd.concat([split, right], axis=1, ignore_index=True) 
# clean by removing quotes and add back header
cleaned = cleaned.apply(lambda col : col.apply(lambda x : x.replace('"', ''))).rename(columns = header)
cleaned.head(5)

,clip_id,no_voice,singer,duet,plucking,hard rock,world,bongos,harpsichord,female singing,...,rap,metal,hip hop,quick,water,baroque,women,fiddle,english,mp3_path
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...
1,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...
2,10,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...
3,11,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...
4,12,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...


### Analyse the dataset

In [10]:
## TODO : Some statistics on the MTT dataset ?
nb_labels_per_song = cleaned.iloc[:,1:-1].astype(int).sum(axis=1)
nb_song_per_label = cleaned.iloc[:,1:-1].astype(int).sum(axis=0)
nb_song_per_label = nb_song_per_label.sort_values(ascending=False)

In [ ]:
# label header is the header without clip_id and mp3_path > length 188 instead of 190
label_header = np.asarray(list(header.values()))[1:-1]
label_header_by_freq = np.asarray(nb_song_per_label.index)

In [ ]:
fig = plt.figure(figsize=(16,6))

plot_nb = 70

y_pos = np.arange(plot_nb)
plt.bar(y_pos, nb_song_per_label[:plot_nb], align='center', alpha=0.5)
plt.xticks(y_pos, nb_song_per_label[:plot_nb])
plt.ylabel('Occurence')
plt.title('Label histogram')
plt.xticks(np.arange(plot_nb), label_header_by_freq[:plot_nb], rotation=90, fontsize = 13)

plt.show()

In [ ]:
# Find the most frequent labels to start with for training
most_freq = nb_song_per_label
for i in range(len(nb_song_per_label)) :
    if nb_song_per_label[i] > 2000:
        print(i, "> ", label_header_by_freq[i], " -- ", nb_song_per_label[i])

In [ ]:
best_10_labels = label_header_by_freq[:10]
best_30_labels = label_header_by_freq[:30]

In [ ]:
print("Number of songs : " , cleaned.shape[0])
print("Number of labels : " , cleaned.shape[1]-2) # -2 is for index columns and mp3 path column
print("Max number of songs tagged with the same label : ",max(nb_song_per_label))
print("Max number of labels for a single song : ",max(nb_labels_per_song))

## Co-occurence matrix

Code from : https://stackoverflow.com/questions/20574257/constructing-a-co-occurrence-matrix-in-python-pandas

In [ ]:
def show_values(pc, fmt="%.2f", **kw):
    '''
    Heatmap with text in each cell with matplotlib's pyplot
    Source: http://stackoverflow.com/a/25074150/395857 
    By HYRY
    '''
    #from itertools import izip
    pc.update_scalarmappable()
    ax = pc.axes
    for p, color, value in zip(pc.get_paths(), pc.get_facecolors(), pc.get_array()):
        x, y = p.vertices[:-2, :].mean(0)
        if np.all(color[:3] > 0.5):
            color = (0.0, 0.0, 0.0)
        else:
            color = (1.0, 1.0, 1.0)
        ax.text(x, y, fmt % value, ha="center", va="center", color=color, **kw)

def cm2inch(*tupl):
    '''
    Specify figure size in centimeter in matplotlib
    Source: http://stackoverflow.com/a/22787457/395857
    By gns-ank
    '''
    inch = 2.54
    if type(tupl[0]) == tuple:
        return tuple(i/inch for i in tupl[0])
    else:
        return tuple(i/inch for i in tupl)

def heatmap(AUC, title, xlabel, ylabel, xticklabels, yticklabels):
    '''
    Inspired by:
    - http://stackoverflow.com/a/16124677/395857 
    - http://stackoverflow.com/a/25074150/395857
    '''

    # Plot it out
    fig, ax = plt.subplots()    
    c = ax.pcolor(AUC, edgecolors='k', linestyle= 'dashed', linewidths=0.2, cmap='RdBu', vmin=0.0, vmax=1.0)

    # put the major ticks at the middle of each cell
    ax.set_yticks(np.arange(AUC.shape[0]) + 0.5, minor=False)
    ax.set_xticks(np.arange(AUC.shape[1]) + 0.5, minor=False)

    # set tick labels
    #ax.set_xticklabels(np.arange(1,AUC.shape[1]+1), minor=False)
    ax.set_xticklabels(xticklabels, minor=False)
    ax.set_yticklabels(yticklabels, minor=False)

    # set title and x/y labels
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)      

    # Remove last blank column
    plt.xlim( (0, AUC.shape[1]) )

    # Turn off all the ticks
    ax = plt.gca()    
    for t in ax.xaxis.get_major_ticks():
        t.tick1On = False
        t.tick2On = False
    for t in ax.yaxis.get_major_ticks():
        t.tick1On = False
        t.tick2On = False

    # Add color bar
    plt.colorbar(c)

    # Add text in each cell 
    show_values(c)

    # Proper orientation (origin at the top left instead of bottom left)
    ax.invert_yaxis()
    ax.xaxis.tick_top()

    # resize 
    fig = plt.gcf()
    fig.set_size_inches(cm2inch(40, 20))



def check_overlaps(label_headers, label_data):


    print('labels:\n{0}'.format(label_data))

    # Compute cooccurrence matrix 
    cooccurrence_matrix = np.dot(label_data.transpose(),label_data)
    print('\ncooccurrence_matrix:\n{0}'.format(cooccurrence_matrix)) 

    # Compute cooccurrence matrix in percentage
    # FYI: http://stackoverflow.com/questions/19602187/numpy-divide-each-row-by-a-vector-element
    #      http://stackoverflow.com/questions/26248654/numpy-return-0-with-divide-by-zero/32106804#32106804
    cooccurrence_matrix_diagonal = np.diagonal(cooccurrence_matrix)
    with np.errstate(divide='ignore', invalid='ignore'):
        cooccurrence_matrix_percentage = np.nan_to_num(np.true_divide(cooccurrence_matrix, \
                                                                      cooccurrence_matrix_diagonal[:, None]))
    print('\ncooccurrence_matrix_percentage:\n{0}'.format(cooccurrence_matrix_percentage))

    # Add count in labels
    label_header_with_count = [ '{0} ({1})'.format(label_header, cooccurrence_matrix_diagonal[label_number]) \
                               for label_number, label_header in enumerate(label_headers)]  
    print('\nlabel_header_with_count: {0}'.format(label_header_with_count))

    # Plotting
    x_axis_size = cooccurrence_matrix_percentage.shape[0]
    y_axis_size = cooccurrence_matrix_percentage.shape[1]
    title = "Co-occurrence matrix\n"
    xlabel= ''#"Labels"
    ylabel= ''#"Labels"
    xticklabels = label_header_with_count
    yticklabels = label_header_with_count
    heatmap(cooccurrence_matrix_percentage, title, xlabel, ylabel, xticklabels, yticklabels)
    plt.savefig('image_output.png', dpi=300, format='png', bbox_inches='tight') 
    # use format='svg' or 'pdf' for vectorial pictures
    #plt.show()

In [ ]:
chosen_labels = best_10_labels
check_overlaps(chosen_labels, cleaned.loc[:,chosen_labels].values.astype(int))

## Load and format mp3 data 

### Create wav files from the mp3 dataset

You should not run this, it was only supposed to be used once for converting all mp3 files.

In [ ]:
def from_mp3_to_wav(files, wav_dir):
    print("Start converting files from mp3 to wav...")
    count = 0
    for file_name in files :
        
        file_name_cut = file_name[len(DATA_DIRECTORY):-4]

        dir_path = wav_dir + file_name_cut[:2]

        new_file_name = wav_dir + file_name_cut + ".wav"
        
        if not os.path.exists(dir_path):
            os.makedirs(dir_path)
            
        try :
            y, sr = librosa.load(file_name, sr=None, mono=True)
            librosa.output.write_wav(new_file_name, y, sr)
            
        except EOFError :
            print("EOFERROR : The following file could not be loaded with librosa - ", file_name) 
            
        count +=1
        if (count % 200) == 0:
            print(count)

In [ ]:
t0a = time.time()
files = find_files(DATA_DIRECTORY)
t1a = time.time()
from_mp3_to_wav(files, WAV_DIRECTORY)
t2a = time.time()

In [ ]:
print("Find files time : {:.3f} sec".format(t1a-t0a))
print("From mp3 to wav time : {:.2f} hours".format((t2a-t1a)/3600))

In [ ]:
print("Time to convert 1 song from mp3 to wav : {:.4f} sec".format((t2a-t1a)/FILE_NB))

### Loading audio & tags

In [24]:
# Use Example 
# load_audio_label(cleaned, num_songs=100, directory=WAV_DIRECTORY, sub_dir=TRAIN_DIR, file_type="wav")

def load_audio_label(labels, num_songs=NB_SONGS, sample_rate=None, directory=WAV_DIRECTORY, \
                     labels_name=LABELS_NAME, sub_dir=None, file_type="wav"):
    
    assert (file_type=="wav" or file_type=="mp3"), "The argument file_type should be either 'wav', either 'mp3'."
    if (file_type=="wav" and directory!=WAV_DIRECTORY) or (file_type=="mp3" and directory!=DATA_DIRECTORY):
        warnings.warn("File type and directory may not correspond. Make sure the \
        directory gave as parameter contains the right type of files", FutureWarning, stacklevel=2)

    if file_type=="mp3" :
        files = find_files(directory, sample=num_songs, sub_dir=sub_dir)
    if file_type=="wav" :
        files = find_files(directory, sample=num_songs, sub_dir=sub_dir, pattern='*.wav')
 
    randomized_files = randomize_files(files)
    
    count = 0
    
    audios = np.ndarray(shape=(num_songs * BATCH_NB, BATCH_SIZE, 1), dtype=np.float32, order='F')
    tags = np.ndarray(shape=(num_songs * BATCH_NB, NB_LABELS), dtype=np.float32, order='F')
    order = np.zeros(num_songs * BATCH_NB, dtype=int)
    
    idx = 0
    
    for filename in randomized_files:

        # Load audio (MP3/WAV) file        
        try :
            audio, _ = librosa.load(filename, sr=sample_rate, mono=True)
        except EOFError :
            print("EOFERROR : The following file could not be loaded with librosa - ", filename)

        audio = audio.reshape(-1, 1)
         
        for n in range(BATCH_NB) :
            audios[idx] = audio[n*BATCH_SIZE: (n+1)*BATCH_SIZE,:]
            
            # take labels or corresponding song
            
            if file_type=="mp3" :
                select_labels  = labels.loc[labels['mp3_path']==filename[len(directory):]]
            
            if file_type=="wav" :
                select_labels  = labels.loc[labels['mp3_path']==filename[len(directory):-4]+".mp3"]

            # select wanted labels
            select_labels = select_labels[labels_name]

            tags[idx] = select_labels.values.reshape(NB_LABELS)
            
            order[idx] = count
            
            idx +=1
        
        count +=1
        if (count % 10) == 0:
            print(count)
    
    return audios, tags, order

In [25]:
# load audios and labels > convert to numpy
# CAREFUL : the argument num_songs is important and shouldn't be too big 
# > otherwise MEMORY ISSUES !!!!!!
print("Loading data ...")
start = time.time()
audios, tags, order = load_audio_label(cleaned, sub_dir=TRAIN_DIR, \
                                file_type="mp3", directory=DATA_DIRECTORY)
end = time.time()
duration = end-start
print(">> Total loading time - {} songs : {:.2f} sec".format(NB_SONGS, duration))

Loading data ...
10
20
>> Total loading time - 20 songs : 0.90 sec


In [26]:
print("Shape of audios list :")
print(audios.shape)
print()
print(">> shape is : [num_song, num_batches, batch_size, 1]")


Shape of audios list :
(180, 51776, 1)

>> shape is : [num_song, num_batches, batch_size, 1]


In [27]:
print("Shape of tags list :")
print(tags.shape)
print()
print(">> shape is : [num_song, num_labels (mp3 file and id excluded), 1] ")
print("(or : [num_song, num_labels, 1, str_len, 1])")

Shape of tags list :
(180, 2)

>> shape is : [num_song, num_labels (mp3 file and id excluded), 1] 
(or : [num_song, num_labels, 1, str_len, 1])


## Neural Network model

### Save neural net state tool

In [ ]:
# not used for now
def save(saver, sess, logdir, step):
    model_name = 'model.ckpt'
    checkpoint_path = os.path.join(logdir, model_name)
    print('Storing checkpoint to {} ...'.format(logdir), end="")
    sys.stdout.flush()

    if not os.path.exists(logdir):
        os.makedirs(logdir)

    saver.save(sess, checkpoint_path, global_step=step)
    print(' Done.')

### Initialize tf

In [ ]:
# run this cell everytime before relaunching tensorflow session
tf.reset_default_graph()

### Training

In [ ]:
# keep results for plotting
train_loss_results = []
train_auc_results = []

In [ ]:
# Lines used to understand local vs global vs trainable
'''print("GLOBAL :", [str(i.name) for i in tf.global_variables()])
    print("LOCAL :", [str(i.name) for i in tf.local_variables()])
    print("TRAINABLE :",[str(i.name) for i in tf.trainable_variables()]) '''

In [ ]:
print("Initializing tf model ...")

audio_tf= tf.convert_to_tensor(audios, np.float32)
print("Input shape : {}".format(audio_tf.shape))
print("Labels : {}".format(tags.flatten()))
print()

net = build_model(audio_tf, is_training=True, config=BASIC_CONFIG) 
predictions = tf.layers.dense(net, NB_LABELS, activation=tf.sigmoid)
loss = tf.nn.sigmoid_cross_entropy_with_logits(labels = tags, logits = predictions)
reduced_loss = tf.reduce_mean(loss)
train_op = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE).minimize(reduced_loss)
auc = tf.metrics.auc(labels = tags, predictions=predictions)


# Saver for storing checkpoints of the model. (Wavenets)
saver = tf.train.Saver(var_list=tf.global_variables())
    
print("Start training...")

start = time.time()

with tf.Session() as sess:
    
    init = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
    
    sess.run(init)
    
    for i in tqdm(range(EPOCHS)):
        predict, _, loss_value, auc_score = sess.run([predictions, train_op, reduced_loss, auc])
        
        auc_result, update_op = auc_score
        
        train_loss_results.append(loss_value)
        train_auc_results.append(auc_result)
        
        print("Iter: {:2}, Loss: {:.4f}, AUC : {:.4f}".format(i, loss_value, auc_result))
        #print("Predictions : {}".format(predict.flatten()))
        #print()
     
    # use wavenet function > see later (for now simplest way)
    #save(saver, sess, LOGDIR, EPOCHS)
    saver.save(sess, LOGDIR)

end = time.time()
duration2 = end-start
print("Total time: {:.2f} sec.".format(duration2))

In [ ]:
# modify this : the epoch axis not not integers which doesn't make sense
fig, axes = plt.subplots(2, sharex=True, figsize=(12, 8))
fig.suptitle('Training Metrics')

axes[0].set_ylabel("Loss", fontsize=14)
axes[0].plot(train_loss_results)

axes[1].set_ylabel("AUC score", fontsize=14)
axes[1].set_xlabel("Epoch", fontsize=14)
axes[1].plot(train_auc_results)
plt.show()

## Test set

In [ ]:
TEST_SIZE = 10

audios_test, tags_test = load_audio_label(cleaned, num_songs=TEST_SIZE, sub_dir=TEST_DIR)

audios_test_tf = tf.convert_to_tensor(audios_test, np.float32)

In [ ]:
variables_to_restore = {
    var.name[:-2]: var for var in tf.global_variables()
    if not ('state_buffer' in var.name or 'pointer' in var.name)}

# Add ops to save and restore all the variables.
saver = tf.train.Saver(variables_to_restore)

# Later, launch the model, use the saver to restore variables from disk, and
# do some work with the model.
with tf.Session() as sess:
      # Restore variables from disk.
    saver.restore(sess, LOGDIR)
    print("Model restored.")
    
    init = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())

    sess.run(init) 

    predicts = sess.run([audios_test_tf, predictions, train_op, reduced_loss, auc])

In [ ]:
# Print some results 
print("AUC Score :", predicts[4])

In [ ]:
print("PREDICTIONS :", predicts[1])
print("TRUE :", tags_test)

Following TF tutorial for custom training
https://github.com/tensorflow/docs/blob/master/site/en/tutorials/eager/custom_training_walkthrough.ipynb